In [1]:
import itertools
from typing import (Any, Dict, FrozenSet, Hashable, ItemsView, Iterator, Iterable,
                    KeysView, List, Mapping, NamedTuple, Optional,
                    Sequence, Tuple, Union, ValuesView, cast)
import random

import numpy as np

from cirq import ops, abc, value, extension, linalg, circuits
import cirq

In [2]:
def ops_to_matrix(*ops):
    c = cirq.Circuit.from_ops(*ops)
    return c.to_unitary_matrix()
q0 = cirq.NamedQubit('q0')
q1 = cirq.NamedQubit('q1')
q2 = cirq.NamedQubit('q2')

In [3]:
np.set_printoptions(precision=3, suppress=4)

In [4]:
ru = cirq.testing.random_unitary

In [5]:
u = ru(4)

In [6]:
cirq.kak_decomposition(u)

((-0.8348877220138199+0.5504202863545045j),
 (array([[-0.203+0.698j, -0.628+0.278j],
         [ 0.628+0.278j, -0.203-0.698j]]),
  array([[-0.926-0.29j ,  0.175+0.167j],
         [-0.175+0.167j, -0.926+0.29j ]])),
 (0.7559864374602536, 0.33871967901847055, -0.0957880786051527),
 (array([[ 0.558-0.188j, -0.22 +0.778j],
         [ 0.22 +0.778j,  0.558+0.188j]]),
  array([[-0.06 +0.781j,  0.457+0.421j],
         [-0.457+0.421j, -0.06 -0.781j]])))

In [7]:
cirq.kak_decomposition(u + 1e-9, tolerance=cirq.Tolerance(atol=1e-4))

((-0.8348877221339872+0.5504202861722323j),
 (array([[-0.203+0.698j, -0.628+0.278j],
         [ 0.628+0.278j, -0.203-0.698j]]),
  array([[-0.926-0.29j ,  0.175+0.167j],
         [-0.175+0.167j, -0.926+0.29j ]])),
 (0.7559864374358813, 0.33871967820331594, -0.09578807922406651),
 (array([[ 0.558-0.188j, -0.22 +0.778j],
         [ 0.22 +0.778j,  0.558+0.188j]]),
  array([[-0.06 +0.781j,  0.457+0.421j],
         [-0.457+0.421j, -0.06 -0.781j]])))

In [8]:
u1, u2, u3, u4 = (ru(2) for _ in range(4))
uu = cirq.dot(cirq.kron(u1, u2), cirq.kron(u3, u4))

In [9]:
cirq.kak_decomposition(uu)

((0.4969290710062269-0.8677911605846699j),
 (array([[-0.422+0.479j,  0.731+0.241j],
         [-0.731+0.241j, -0.422-0.479j]]),
  array([[ 0.825+0.326j, -0.43 +0.167j],
         [ 0.43 +0.167j,  0.825-0.326j]])),
 (0.0, 0.0, 0.0),
 (array([[ 0.293-0.017j,  0.951+0.094j],
         [-0.951+0.094j,  0.293+0.017j]]),
  array([[-0.586+0.683j, -0.293+0.322j],
         [ 0.293+0.322j, -0.586-0.683j]])))

In [10]:
u1, u2, u3, u4

(array([[-0.266+0.606j, -0.117+0.74j ],
        [ 0.598-0.452j, -0.409+0.52j ]]),
 array([[ 0.444-0.712j, -0.024-0.543j],
        [-0.497+0.22j ,  0.441-0.714j]]),
 array([[ 0.723+0.575j, -0.346-0.165j],
        [ 0.382+0.039j,  0.916-0.116j]]),
 array([[-0.681-0.509j,  0.268+0.454j],
        [-0.484-0.209j, -0.59 -0.612j]]))

In [11]:
random.random()

0.17275456675109813

In [12]:
c = cirq.Circuit.from_ops(
    cirq.google.decompositions._xx_yy_zz_interaction_via_full_czs(q0, q1,
        random.random()*np.pi/4, random.random()*np.pi/4, random.random()*np.pi/4)
        #0.61, 0.31, 0.21)
)
o = c.to_unitary_matrix()
c

q0: ───X^0.5───@───X^0.0352───H─────────@───H──────────────────────@───────
               │                        │                          │
q1: ───H───────@───H──────────Y^0.409───@───X^-0.5───Z^0.222───H───@───H───

In [13]:
cirq.kak_decomposition(c.to_unitary_matrix())

((0.9661777356674771+0.2578770697453862j),
 (array([[ 0.   -0.707j, -0.707+0.j   ],
         [ 0.707+0.j   ,  0.   +0.707j]]),
  array([[ 0.   +0.707j,  0.707+0.j   ],
         [-0.707+0.j   ,  0.   -0.707j]])),
 (0.7300843115172415, 0.4372308362950137, 0.14265686860796156),
 (array([[ 0.   -0.707j, -0.707+0.j   ],
         [ 0.707+0.j   ,  0.   +0.707j]]),
  array([[ 0.   +0.707j,  0.707+0.j   ],
         [-0.707+0.j   ,  0.   -0.707j]])))

In [14]:
c2 = cirq.Circuit.from_ops(
    cirq.google.two_qubit_matrix_to_native_gates(q0, q1, c.to_unitary_matrix(),
                                                 allow_partial_czs=False)
)
o2 = c2.to_unitary_matrix()
c2

q0: ───X^0.5───Z───────X^0.5───────@───X^0.0352───H─────────@───H──────────────────────@───X^0.5───Z───────────
                                   │                        │                          │
q1: ───────────X^0.5───Z───────H───@───H──────────Y^0.222───@───X^-0.5───Z^0.409───H───@───H───────X^0.5───Z───

In [15]:
cirq.allclose_up_to_global_phase(o, o2)

True

In [16]:
cirq.kak_decomposition(
    cirq.Circuit.from_ops(
        cirq.X(q0) ** .5,
        cirq.X(q1) ** 0,
        cirq.google.decompositions._xx_yy_zz_interaction_via_full_czs(q0, q1,
        random.random()*2*np.pi, random.random()*2*np.pi, random.random()*2*np.pi)
    ).to_unitary_matrix()
)

((-0.4913429530741206+0.8709661890477738j),
 (array([[ 0.   +0.j   ,  0.707-0.707j],
         [-0.707-0.707j,  0.   -0.j   ]]),
  array([[-0.707-0.707j,  0.   -0.j   ],
         [-0.   -0.j   , -0.707+0.707j]])),
 (0.7808863958990586, 0.42899080566244274, 0.0801284221861891),
 (array([[ 0.5+0.5j,  0.5-0.5j],
         [-0.5-0.5j,  0.5-0.5j]]), array([[ 0.   +0.j   ,  0.707+0.707j],
         [-0.707+0.707j,  0.   -0.j   ]])))

In [43]:
c = cirq.Circuit.from_ops(
    cirq.google.two_qubit_matrix_to_native_gates(q0, q1,
        cirq.Circuit.from_ops(cirq.CZ(q0,q1)**.1,cirq.X(q0)**.5,cirq.X(q1)**0).to_unitary_matrix(),
                                                 allow_partial_czs=False)
)
c

q0: ───X^-0.5───S^-1────────────────@───X^-0.05───@───X^-0.95───S^-1───────────────
                                    │             │
q1: ────────────X^0.5───Z^0.5───H───@─────────────@───H─────────Y^-0.5───Z^-0.45───

In [44]:
cirq.google.optimized_for_xmon(c)

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]] [GateOperation(Exp11Gate(half_turns=1.0), (NamedQubit('q0'), NamedQubit('q1')))]
[[-0.036-0.042j -0.036-0.042j  0.536-0.458j -0.536+0.458j]
 [ 0.036+0.042j -0.036-0.042j -0.536+0.458j -0.536+0.458j]
 [ 0.536-0.458j  0.536-0.458j -0.036-0.042j  0.036+0.042j]
 [-0.536+0.458j  0.536-0.458j  0.036+0.042j  0.036+0.042j]] [GateOperation(Exp11Gate(half_turns=1.0), (NamedQubit('q0'), NamedQubit('q1'))), GateOperation(ExpWGate(half_turns=-0.9500000000000002, axis_half_turns=0.0), (NamedQubit('q0'),)), GateOperation(ExpWGate(half_turns=-0.5, axis_half_turns=0.5), (NamedQubit('q1'),))]


q0: ───X^-0.5───@───Y^-0.05───@───Y^-0.95───Z───
                │             │
q1: ────────────@─────────────@───Z^0.05────────

In [18]:
cc = cirq.Circuit.from_ops(cirq.CZ(q0, q1), cirq.CZ(q0, q1), cirq.X(q0)**.5,
                           cirq.X(q1)**0.0000000000001)
cc

q0: ───@───@───X^0.5────────
       │   │
q1: ───@───@───X^9.99e-14───

In [19]:
cirq.google.MergeInteractions().optimize_circuit(cc)
cc

[[ 0.5+0.5j  0. -0.j   0.5-0.5j -0. -0.j ]
 [ 0. -0.j   0.5+0.5j -0. -0.j   0.5-0.5j]
 [ 0.5-0.5j -0. -0.j   0.5+0.5j  0. -0.j ]
 [-0. -0.j   0.5-0.5j  0. -0.j   0.5+0.5j]] [GateOperation(CZ, (NamedQubit('q0'), NamedQubit('q1'))), GateOperation(CZ, (NamedQubit('q0'), NamedQubit('q1'))), GateOperation(X**0.5, (NamedQubit('q0'),)), GateOperation(X**9.992007221626409e-14, (NamedQubit('q1'),))]


q0: ───W(0.981)^-0.316───Z^0.121────W(0.137)^0.185───Z^-0.108───

q1: ───W(0.876)^-0.52────Z^-0.274───W(0.602)^0.52────Z^0.274────

In [20]:
(cirq.X**0).matrix()[1,1]-1

0j

In [21]:
c_ = cirq.Circuit.from_ops(
    cirq.google.decompositions._xx_yy_zz_interaction_via_full_czs(q0, q1,
        #random.random()*np.pi/4, random.random()*np.pi/4, random.random()*np.pi/4)
        .25*np.pi, .25*np.pi, .75*np.pi)
        #0.61, 0.31, 0.21)
)
display(c)
xyz = c_.to_unitary_matrix()
xyz

q0: ───X^0.5───@───X^0.0352───H─────────@───H──────────────────────@───────
               │                        │                          │
q1: ───H───────@───H──────────Y^0.409───@───X^-0.5───Z^0.222───H───@───H───

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]])

In [22]:
u2 = ops_to_matrix(cirq.Z(q0)**.5)
cirq.dot(
    cirq.kron(u2, u2),
    xyz,
    cirq.kron(u2, u2),
)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [23]:
cirq.kak_canonicalize_vector(0.00000000000001, 0.0000000000001, 0.0000000001)

((1+0j), (array([[0.+0.707j, 0.-0.707j],
         [0.-0.707j, 0.-0.707j]]), array([[0.+0.707j, 0.-0.707j],
         [0.-0.707j, 0.-0.707j]])), (1e-10,
  1e-13,
  1e-14), (array([[0.+0.707j, 0.-0.707j],
         [0.-0.707j, 0.-0.707j]]), array([[0.+0.707j, 0.-0.707j],
         [0.-0.707j, 0.-0.707j]])))

In [24]:
def random_x_y_z():
    x = random.random() * 2 * np.pi
    y = random.random() * 2 * np.pi
    z = random.random() * 2 * np.pi
    return x, y, z
random_x_y_z()

(5.242903840816487, 3.9324285819855627, 1.046255600576676)

In [32]:
def f():
    x, y, z = random_x_y_z()
    g, (cl1, cl0), (x2, y2, z2), (cr1, cr0) = cirq.kak_canonicalize_vector(x, y, z)
    rl1, rl0, rr1, rr0 = (cirq.testing.random_unitary(2) for _ in range(4))
    l1 = cirq.dot(cl1, rl1)
    l0 = cirq.dot(cl0, rl0)
    r1 = cirq.dot(cr1, rr1)
    r0 = cirq.dot(cr0, rr0)
    m = cirq.dot(
        cirq.kron(l0, l1),
        ops_to_matrix(cirq.google.decompositions._xx_yy_zz_interaction_via_full_czs(
                            q0, q1, x2, y2, z2)),
        cirq.kron(r0, r1)
    )
    gg, (ll1, ll0), (xx, yy, zz), (rr1, rr0) = cirq.kak_decomposition(m)
    return x2, xx, y2, yy, z2, zz

In [33]:
f()

(0.577872625630008,
 0.577872625630008,
 0.35571206549864076,
 0.35571206549864093,
 0.32522956953227,
 0.32522956953227006)

In [27]:
ops_to_matrix(
    cirq.google.decompositions._xx_yy_zz_interaction_via_full_czs(q0, q1,
        random.random()*np.pi/4, random.random()*np.pi/4, random.random()*np.pi/4))

array([[ 0.747+0.556j,  0.   +0.j   ,  0.   +0.j   ,  0.218-0.293j],
       [ 0.   +0.j   ,  0.545+0.243j, -0.327+0.733j,  0.   +0.j   ],
       [ 0.   +0.j   , -0.327+0.733j,  0.545+0.243j,  0.   +0.j   ],
       [ 0.218-0.293j,  0.   +0.j   ,  0.   +0.j   ,  0.747+0.556j]])

In [36]:
c_ = cirq.Circuit.from_ops(
cirq.google.decompositions._xx_interaction_via_full_czs(q0, q1, .2*np.pi))
display(c_)
xy = c_.to_unitary_matrix()
xy

q0: ───────@───X^-0.4───@───────
           │            │
q1: ───H───@────────────@───H───

array([[0.655-0.476j, 0.   +0.j   , 0.   +0.j   , 0.345+0.476j],
       [0.   +0.j   , 0.655-0.476j, 0.345+0.476j, 0.   +0.j   ],
       [0.   +0.j   , 0.345+0.476j, 0.655-0.476j, 0.   +0.j   ],
       [0.345+0.476j, 0.   +0.j   , 0.   +0.j   , 0.655-0.476j]])

In [29]:
u2 = ops_to_matrix(cirq.X(q0))
cirq.dot(
    cirq.kron(u2, u2),
    xy,
    cirq.kron(u2, u2),
)

array([[ 0.042-0.997j,  0.   +0.j   ,  0.   +0.j   , -0.063-0.003j],
       [ 0.   +0.j   ,  0.002-0.042j,  0.998+0.042j,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.998+0.042j,  0.002-0.042j,  0.   +0.j   ],
       [-0.063-0.003j,  0.   +0.j   ,  0.   +0.j   ,  0.042-0.997j]])